In [1]:
from utils.util import load_from_txt,discard_overlaps,iou
import cv2
from collections import defaultdict

In [2]:
def frame_visualization(video_capture,predicted_boxes,frame_id,out_path,method):
    video_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_id - 1)
    res, frame = video_capture.read()
    # Draw the ground truth boxes
    # Draw the predicted boxes
    for box in predicted_boxes[frame_id]:
        cv2.rectangle(frame, (int(box[1]), int(box[2])), (int(box[3]), int(box[4])), (0, 0, 255), 2)
        cv2.putText(frame, f'id:{box[6]}',(int(box[1]), int(box[2])), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255),2)
    # put text mIOU of frame
    # put text number of frame
    cv2.putText(frame, f"Frame: {frame_id}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imwrite(f'{out_path}/{method}_{frame_id}.png', frame)

    ret, frame = video_capture.read()

In [3]:
mask_rcnn = 'C:/Users/AnaHarris/Documents/MASTER/M6/project/dataset/AICity_data/train/S03/c010/det/det_mask_rcnn.txt'
ssd512 =  'C:/Users/AnaHarris/Documents/MASTER/M6/project/dataset/AICity_data/train/S03/c010/det/det_ssd512.txt'
yolo3 =  'C:/Users/AnaHarris/Documents/MASTER/M6/project/dataset/AICity_data/train/S03/c010/det/det_yolo3.txt'

In [4]:
video_capture = cv2.VideoCapture('C:/Users/AnaHarris/Documents/MASTER/M6/project/dataset/AICity_data/train/S03/c010/vdo.avi')

In [5]:
mask_rcnn_boxes = load_from_txt(mask_rcnn,threshold=0.5)

In [6]:
track_id = 0
tracked_objects = defaultdict(list)
for frame_id in mask_rcnn_boxes:
    # REMOVE OVERLAPPING BOUNDING BOXES
    boxes = mask_rcnn_boxes[frame_id]
    frame_boxes = discard_overlaps(boxes)
    # FIRST FRAME, WE INITIALIZE THE OBJECTS ID
    if frame_id == 0:
        for j in range(len(frame_boxes)):
            # We add the tracking object ID at the end of the list  [[frame,x1, y1, x2, y2, conf, track_id]]
            frame_boxes[j].append(track_id)
            tracked_objects[f't_id_{track_id}'].append([frame_boxes[j][1],frame_boxes[j][2],frame_boxes[j][3],frame_boxes[j][4]])
            track_id += 1
            
    else:
        # FRAME N+1 WE COMPARE TO OBJECTS IN FRAME N
        for i in range(len(frame_boxes)):
            best_iou = 0
            track_id_best = 0
            boxA = [frame_boxes[i][1],frame_boxes[i][2],frame_boxes[i][3],frame_boxes[i][4]]

            for j in range(len(boxes_previous_frame)):
                boxB = [boxes_previous_frame[j][1],boxes_previous_frame[j][2],boxes_previous_frame[j][3],boxes_previous_frame[j][4]]
                iou_score,_ = iou(boxA,boxB)

                if iou_score > best_iou:
                    best_iou = iou_score
                    track_id_best = boxes_previous_frame[j][6]

            if track_id_best == 0 and best_iou == 0:
                track_id += 1
                frame_boxes[i].append(track_id)
                frame_boxes[i].append(best_iou)
           
            else:
                # CHECK IF THERE ARE WRONG TRACKED OBJECTS (we cannot have more than 1 object with the same ID)
                if any(track_id_best in sublist for sublist in frame_boxes):
                    wrong_tracked = [sublist  for sublist in frame_boxes if track_id_best in sublist][0]
                    if best_iou > wrong_tracked[-1]:
                        frame_boxes[i].append(track_id_best)
                        frame_boxes[i].append(best_iou)

                        wrong_tracked[-2] = track_id
                        wrong_tracked[-1] = 0
                        track_id += 1
                    
                    else:
                        frame_boxes[i].append(track_id)
                        frame_boxes[i].append(0)
                        track_id += 1
                
                else:
                    frame_boxes[i].append(track_id_best)
                    frame_boxes[i].append(best_iou)


    """ tracked_objects[f't_id_{track_id}'].append(boxA)
    tracked_objects[f't_id_{track_id_best}'].append(boxA) """
    boxes_previous_frame = frame_boxes


In [7]:
for i in range(0,len(mask_rcnn_boxes),10):
    frame_visualization(video_capture,mask_rcnn_boxes,i,'C:/Users/AnaHarris/Documents/GitHub/mcv-m6-2023-team6/week3/Results/task2_1','mask_rcnn')